# context v try 2

In [22]:
#TRY 2

In [1]:
import pandas as pd
import numpy as np
spec_path=r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\data\train_specifications.csv"
csv_path = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\super_same_norm.csv"

# Your original sensor features
sensor_features =[
    '171_0', '666_0', '427_0', '837_0', '167_0', '167_1', '167_2', '167_3', '167_4', '167_5', '167_6', '167_7',
    '167_8', '167_9', '309_0', '272_0', '272_1', '272_2', '272_3', '272_4', '272_5', '272_6', '272_7', '272_8', '272_9',
    '835_0', '370_0', '291_0', '291_1', '291_2', '291_3', '291_4', '291_5', '291_6', '291_7', '291_8', '291_9', '291_10',
    '158_0', '158_1', '158_2', '158_3', '158_4', '158_5', '158_6', '158_7', '158_8', '158_9', '100_0',
    '459_0', '459_1', '459_2', '459_3', '459_4', '459_5', '459_6', '459_7', '459_8', '459_9', '459_10', '459_11',
    '459_12', '459_13', '459_14', '459_15', '459_16', '459_17', '459_18', '459_19',
    '397_0', '397_1', '397_2', '397_3', '397_4', '397_5', '397_6', '397_7', '397_8', '397_9',
    '397_10', '397_11', '397_12', '397_13', '397_14', '397_15', '397_16', '397_17', '397_18', '397_19',
    '397_20', '397_21', '397_22', '397_23', '397_24', '397_25', '397_26', '397_27', '397_28', '397_29',
    '397_30', '397_31', '397_32', '397_33', '397_34', '397_35'
]
def generate_context_vectors(csv_path, spec_path, context=70, verbose=True):
    df = pd.read_csv(csv_path)
    specs = pd.read_csv(spec_path)
    
    context_vectors = []
    target_rul = []
    vehicle_ids = []

    for vehicle_id, group in df.groupby('vehicle_id'):
        group = group.sort_values('time_step')
        data = group[sensor_features]

        if len(data) < context:
            if verbose:
                print(f"Skipping vehicle_id {vehicle_id} due to insufficient length ({len(data)}<{context})")
            continue

        # Compute summary statistics for the entire vehicle sequence
        stats = []
        stats.append(data.mean().values)    # Mean of each feature
        stats.append(data.std().values)     # Std dev of each feature
        stats.append(data.min().values)     # Min of each feature
        stats.append(data.max().values)     # Max of each feature
        stats.append(data.iloc[-1].values)  # Last time-step

        context_vector = np.concatenate(stats)
        context_vectors.append(context_vector)

        # Use the last RUL as target (or any other RUL definition)
        target_rul.append(group['RUL'].values[-1])
        vehicle_ids.append(vehicle_id)

    # Create DataFrame
    context_df = pd.DataFrame(context_vectors)
    context_df['vehicle_id'] = vehicle_ids
    context_df['RUL'] = target_rul

    # Merge with specs
    final_df = pd.merge(context_df, specs, on='vehicle_id', how='inner')

    return final_df

# Example usage:
# csv_path = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\super_same_norm.csv"
# spec_path = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\specifications.csv"

final_tabular_df = generate_context_vectors(csv_path, spec_path)

# Save to CSV (optional)
final_tabular_df.to_csv("tabtransformer_input.csv", index=False)


Skipping vehicle_id 22 due to insufficient length (51<70)
Skipping vehicle_id 29 due to insufficient length (44<70)
Skipping vehicle_id 67 due to insufficient length (57<70)
Skipping vehicle_id 75 due to insufficient length (56<70)
Skipping vehicle_id 79 due to insufficient length (38<70)
Skipping vehicle_id 95 due to insufficient length (32<70)
Skipping vehicle_id 186 due to insufficient length (9<70)
Skipping vehicle_id 246 due to insufficient length (66<70)
Skipping vehicle_id 336 due to insufficient length (61<70)
Skipping vehicle_id 351 due to insufficient length (69<70)
Skipping vehicle_id 358 due to insufficient length (30<70)
Skipping vehicle_id 368 due to insufficient length (27<70)
Skipping vehicle_id 370 due to insufficient length (66<70)
Skipping vehicle_id 386 due to insufficient length (48<70)
Skipping vehicle_id 410 due to insufficient length (43<70)
Skipping vehicle_id 430 due to insufficient length (68<70)
Skipping vehicle_id 457 due to insufficient length (19<70)
Skip

In [ ]:
import torch
import torch.nn as nn

class TransformerRegressor(nn.Module):
    def __init__(self, input_dim, model_dim=128, num_heads=4, num_layers=2, dropout=0.1):
        super(TransformerRegressor, self).__init__()

        self.model_dim = model_dim

        self.input_projection = nn.Linear(input_dim, model_dim)
        self.positional_encoding = PositionalEncoding(model_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.regressor = nn.Sequential(
            nn.Linear(model_dim, 64), 
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.input_projection(x)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        # Use the last timestep's output
        return self.regressor(x[:, -1, :])

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model).float()
        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


In [27]:
import pandas as pd
import numpy as np

sensor_features = [
    '171_0', '666_0', '427_0', '837_0', '167_0', '167_1', '167_2', '167_3', '167_4', '167_5', '167_6', '167_7',
    '167_8', '167_9', '309_0', '272_0', '272_1', '272_2', '272_3', '272_4', '272_5', '272_6', '272_7', '272_8', '272_9',
    '835_0', '370_0', '291_0', '291_1', '291_2', '291_3', '291_4', '291_5', '291_6', '291_7', '291_8', '291_9', '291_10',
    '158_0', '158_1', '158_2', '158_3', '158_4', '158_5', '158_6', '158_7', '158_8', '158_9', '100_0',
    '459_0', '459_1', '459_2', '459_3', '459_4', '459_5', '459_6', '459_7', '459_8', '459_9', '459_10', '459_11',
    '459_12', '459_13', '459_14', '459_15', '459_16', '459_17', '459_18', '459_19',
    '397_0', '397_1', '397_2', '397_3', '397_4', '397_5', '397_6', '397_7', '397_8', '397_9',
    '397_10', '397_11', '397_12', '397_13', '397_14', '397_15', '397_16', '397_17', '397_18', '397_19',
    '397_20', '397_21', '397_22', '397_23', '397_24', '397_25', '397_26', '397_27', '397_28', '397_29',
    '397_30', '397_31', '397_32', '397_33', '397_34', '397_35'
]

def generate_context_vectors(csv_path, context=70):
    df = pd.read_csv(csv_path)
    X_list, rul_list, id_list = [], [], []

    for vehicle_id, group in df.groupby('vehicle_id'):
        group = group.sort_values('time_step')
        data = group[sensor_features].values
        rul = group['RUL'].values

        if len(data) < context:
            continue

        for i in range(len(data) - context + 1):
            context_vector = data[i:i+context].mean(axis=0)
            X_list.append(context_vector)
            rul_list.append(rul[i + context - 1])
            id_list.append(vehicle_id)

    context_df = pd.DataFrame(X_list, columns=[f"feat_{i}" for i in range(len(sensor_features))])
    context_df['vehicle_id'] = id_list
    context_df['RUL'] = rul_list
    return context_df


In [28]:
from sklearn.preprocessing import OrdinalEncoder

def prepare_tabtransformer_input(context_df, spec_path):
    specs = pd.read_csv(spec_path)

    # Normalize column names
    specs.columns = specs.columns.str.strip()
    context_df.columns = context_df.columns.str.strip()

    # Identify spec columns
    spec_cols = [col for col in specs.columns if col.startswith("Spec_")]
    assert 'vehicle_id' in specs.columns, "'vehicle_id' not found in spec file"

    # Merge with context vectors
    merged = pd.merge(context_df, specs, on='vehicle_id', how='inner')

    # Encode categorical specs
    encoder = OrdinalEncoder()
    cat_specs = encoder.fit_transform(merged[spec_cols].astype(str))
    cat_df = pd.DataFrame(cat_specs, columns=spec_cols)

    # Final X and y
    numeric_features = [col for col in context_df.columns if col.startswith("feat_")]
    final_X = pd.concat([merged[numeric_features].reset_index(drop=True), cat_df], axis=1)
    y = merged['RUL'].values
    return final_X, y, spec_cols, encoder


In [29]:
import torch
from torch.utils.data import Dataset, DataLoader

class TabTransformerDataset(Dataset):
    def __init__(self, X, y, cat_indices):
        self.X = X
        self.y = y
        self.cat_idx = cat_indices
        self.cont_idx = [i for i in range(X.shape[1]) if i not in cat_indices]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X.iloc[idx].values
        cont = torch.tensor(x[self.cont_idx], dtype=torch.float32)
        cat = torch.tensor(x[self.cat_idx], dtype=torch.long)
        label = torch.tensor(self.y[idx], dtype=torch.float32)
        return cat, cont, label


In [30]:
import torch.nn as nn

class TabTransformer(nn.Module):
    def __init__(self, num_cont, categories, d_token=32):
        super().__init__()
        self.cat_embeds = nn.ModuleList([
            nn.Embedding(cat_size, d_token) for cat_size in categories
        ])
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_token, nhead=4),
            num_layers=2
        )
        self.cont_norm = nn.BatchNorm1d(num_cont)
        self.head = nn.Sequential(
            nn.Linear(d_token * len(categories) + num_cont, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x_cat, x_cont):
        x_cat = torch.stack([emb(x_cat[:, i]) for i, emb in enumerate(self.cat_embeds)], dim=1)
        x_cat = self.transformer(x_cat).flatten(1)
        x_cont = self.cont_norm(x_cont)
        x = torch.cat([x_cat, x_cont], dim=1)
        return self.head(x).squeeze(1)


In [ ]:
def train_model(model, train_loader, val_loader=None, epochs=10, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for cat, cont, y in train_loader:
            cat, cont, y = cat.to(device), cont.to(device), y.to(device)
            optimizer.zero_grad()
            preds = model(cat, cont)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {total_loss/len(train_loader):.4f}")
        


In [32]:
from sklearn.model_selection import train_test_split

# File paths
csv_path = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\super_same_norm.csv"
spec_path = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\data\train_specifications.csv"

# Generate context vectors
context_df = generate_context_vectors(csv_path)

# Prepare inputs
X, y, spec_cols, encoder = prepare_tabtransformer_input(context_df, spec_path)

# Train/Val split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
cat_indices = list(range(X.shape[1] - len(spec_cols), X.shape[1]))
category_sizes = [len(cats) for cats in encoder.categories_]

train_ds = TabTransformerDataset(X_train, y_train, cat_indices)
val_ds = TabTransformerDataset(X_val, y_val, cat_indices)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

# Train model
model = TabTransformer(num_cont=X.shape[1] - len(spec_cols), categories=category_sizes)
train_model(model, train_loader, val_loader)


C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/10 - Train Loss: 373.0049
Epoch 2/10 - Train Loss: 266.4913
Epoch 3/10 - Train Loss: 201.8929
Epoch 4/10 - Train Loss: 151.9971
Epoch 5/10 - Train Loss: 121.6993
Epoch 6/10 - Train Loss: 99.6516
Epoch 7/10 - Train Loss: 87.5982
Epoch 8/10 - Train Loss: 85.3491
Epoch 9/10 - Train Loss: 77.0060
Epoch 10/10 - Train Loss: 69.9276


# working

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OrdinalEncoder

# --- 1. Load your sequence windows and labels ---
def create_X_y(csv_path, sensor_features, context=70, verbose=True):
    df = pd.read_csv(csv_path)
    X, y, vid = [], [], []
    for vehicle_id, group in df.groupby('vehicle_id'):
        group = group.sort_values('time_step')
        data = group[sensor_features].values
        rul = group['RUL'].values
        if len(data) < context:
            if verbose:
                print(f"Skipping vehicle {vehicle_id}: {len(data)}<{context}")
            continue
        for i in range(len(data) - context + 1):
            X.append(data[i:i+context])
            y.append(rul[i+context-1])
            vid.append(vehicle_id)
    X = np.stack(X)          # 6553,70,105  # shape (N, context, num_sensors)
    y = np.array(y)      #6553,     # shape (N,)
    vid = np.array(vid)  #6533,     # shape (N,)
    print(f"Total windows: {len(X)}, window shape: {X.shape[1:]}")
    return X, y, vid

# your sensor_features list as above...
sensor_features = ['171_0', '666_0', '427_0', '837_0', '167_0', '167_1', '167_2', '167_3', '167_4', '167_5', '167_6', '167_7',     '167_8', '167_9', '309_0', '272_0', '272_1', '272_2', '272_3', '272_4', '272_5', '272_6', '272_7', '272_8', '272_9',     '835_0', '370_0', '291_0', '291_1', '291_2', '291_3', '291_4', '291_5', '291_6', '291_7', '291_8', '291_9', '291_10',     '158_0', '158_1', '158_2', '158_3', '158_4', '158_5', '158_6', '158_7', '158_8', '158_9', '100_0',     '459_0', '459_1', '459_2', '459_3', '459_4', '459_5', '459_6', '459_7', '459_8', '459_9', '459_10', '459_11',     '459_12', '459_13', '459_14', '459_15', '459_16', '459_17', '459_18', '459_19',     '397_0', '397_1', '397_2', '397_3', '397_4', '397_5', '397_6', '397_7', '397_8', '397_9',     '397_10', '397_11', '397_12', '397_13', '397_14', '397_15', '397_16', '397_17', '397_18', '397_19',     '397_20', '397_21', '397_22', '397_23', '397_24', '397_25', '397_26', '397_27', '397_28', '397_29',     '397_30', '397_31', '397_32', '397_33', '397_34', '397_35' ] 

csv_path = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\super_same_norm.csv"
X, y, vids = create_X_y(csv_path, sensor_features, context=70)

# --- 2. Define a PyTorch Dataset & Transformer‐based embedder ---
class WindowDataset(Dataset):
    def __init__(self, X, vids):
        self.X = torch.from_numpy(X).float()   #6553,70,105        # (N, context, F)
        self.vids = vids
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.vids[idx]

class TimeSeriesEmbedder(nn.Module):
    def __init__(self, num_features, d_model=128, n_heads=8, num_layers=2, dropout=0.1):
        super().__init__()
        # project input features to d_model                                 #num_features=len(sensor_features)=105
        self.input_proj = nn.Linear(num_features, d_model)                  #  105,128
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,         #
                                                   nhead=n_heads,
                                                   dropout=dropout,
                                                   batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # we'll use mean‐pooling over the context dimension
    def forward(self, x):
        """
        x: (batch, context, num_features)
        returns: (batch, d_model)
        """                             #i choose 256 batches
        x = self.input_proj(x)          #256,70,128     # → (batch, context, d_model)
        x = self.encoder(x)             #256,70,128     # → (batch, context, d_model)
        return x.mean(dim=1)            #256,128     # → (batch, d_model)

# instantiate
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedder = TimeSeriesEmbedder(num_features=len(sensor_features), d_model=128).to(device)
embedder.eval()  # inference mode


Skipping vehicle 22: 51<70
Skipping vehicle 29: 44<70
Skipping vehicle 67: 57<70
Skipping vehicle 75: 56<70
Skipping vehicle 79: 38<70
Skipping vehicle 95: 32<70
Skipping vehicle 186: 9<70
Skipping vehicle 246: 66<70
Skipping vehicle 336: 61<70
Skipping vehicle 351: 69<70
Skipping vehicle 358: 30<70
Skipping vehicle 368: 27<70
Skipping vehicle 370: 66<70
Skipping vehicle 386: 48<70
Skipping vehicle 410: 43<70
Skipping vehicle 430: 68<70
Skipping vehicle 457: 19<70
Skipping vehicle 467: 45<70
Skipping vehicle 482: 23<70
Skipping vehicle 485: 61<70
Skipping vehicle 522: 47<70
Skipping vehicle 552: 60<70
Skipping vehicle 563: 15<70
Skipping vehicle 629: 50<70
Skipping vehicle 638: 33<70
Skipping vehicle 646: 66<70
Skipping vehicle 657: 46<70
Skipping vehicle 682: 66<70
Skipping vehicle 693: 8<70
Skipping vehicle 734: 16<70
Skipping vehicle 738: 51<70
Skipping vehicle 744: 39<70
Skipping vehicle 760: 67<70
Skipping vehicle 779: 59<70
Skipping vehicle 797: 8<70
Skipping vehicle 817: 25<70
S

TimeSeriesEmbedder(
  (input_proj): Linear(in_features=105, out_features=128, bias=True)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
)

In [ ]:
# --- 3. Batch‐wise embedding generation ---
dataset = WindowDataset(X, vids)
loader  = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=0)

all_embs = []
all_vids = []
with torch.no_grad():
    for batch_x, batch_vid in loader:
        batch_x = batch_x.to(device)             #batch=256
        embs = embedder(batch_x)                 # (batch, 128)
        all_embs.append(embs.cpu().numpy())
        all_vids.append(batch_vid)

all_embs = np.vstack(all_embs)    # (N, 128)
all_vids = np.concatenate(all_vids)

# Save embeddings and vids so you can reload later
np.save("sequence_embeddings.npy", all_embs)
np.save("sequence_vehicle_ids.npy", all_vids)

# --- 4. Load & encode your vehicle specifications ---
spec_df = pd.read_csv(
    r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\data\train_specifications.csv"
)
# assume spec_df has columns: ['vehicle_id', 'Spec_0', …, 'Spec_7']
cats = [f"Spec_{i}" for i in range(8)]
encoder = OrdinalEncoder()
spec_df[cats] = encoder.fit_transform(spec_df[cats])

# Repeat specs for each window:
specs_per_window = pd.DataFrame({
    'vehicle_id': all_vids
}).merge(spec_df, on='vehicle_id', how='left')[cats].values  # shape (N, 8)-->cat name ka column create

# --- 5. Final feature matrix for TabTransformer ---
# Combine your 128‐dim time‐series embedding + 8 spec columns → 136 dims
X_tab = np.hstack([all_embs, specs_per_window])  # shape (N, 136)
y_tab = np.load("sequence_labels.npy") if os.path.exists("sequence_labels.npy") else y





# a = np.array([1, 2, 3])
# b = np.array([4, 5, 6])

# result = np.hstack((a, b))
# print(result)  # Output: [1 2 3 4 5 6]



# a = np.array([[1], [2], [3]])
# b = np.array([[4], [5], [6]])

# result = np.hstack((a, b))
# print(result)
# # Output:
# # [[1 4]
# #  [2 5]
# #  [3 6]]









# Save or wrap into a Dataset for TabTransformer training:
np.save("X_tab.npy", X_tab)
np.save("y_tab.npy", y_tab)
print("Saved X_tab.npy (shape:", X_tab.shape, ") and y_tab.npy (shape:", y_tab.shape, ")")


Saved X_tab.npy (shape: (6553, 136) ) and y_tab.npy (shape: (6553,) )


In [ ]:
# np.save("y.npy", y)

In [ ]:
np.save("y.npy", y)
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tab_transformer_pytorch import TabTransformer

# Configuration
EMBEDDINGS_PATH = 'sequence_embeddings.npy'
VEHICLE_IDS_PATH = 'sequence_vehicle_ids.npy'
SPEC_CSV_PATH = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\data\train_specifications.csv"
LABELS_PATH = 'y.npy'  # if already saved, otherwise use y from create_X_y

BATCH_SIZE = 256
NUM_EPOCHS = 20
LEARNING_RATE = 1e-3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Load embeddings and vehicle IDs
embs = np.load(EMBEDDINGS_PATH)      # shape (N, E)
vids = np.load(VEHICLE_IDS_PATH)      # shape (N,)
labels = np.load(LABELS_PATH)         # shape (N,)

# 2. Load and encode specifications
spec_df = pd.read_csv(SPEC_CSV_PATH)
cats = [f'Spec_{i}' for i in range(8)]
encoder = OrdinalEncoder()
spec_df[cats] = encoder.fit_transform(spec_df[cats])

# Merge specs for each window
specs = pd.DataFrame({'vehicle_id': vids}).merge(
    spec_df[['vehicle_id'] + cats], on='vehicle_id', how='left'
)[cats].values

# 3. Prepare continuous features (embeddings) with scaler
scaler = StandardScaler()
cont = scaler.fit_transform(embs)    # will also be passed as continuous_mean_std
cont_mean_std = torch.tensor(np.stack([scaler.mean_, scaler.scale_], axis=1), dtype=torch.float)

# - scaler.fit_transform(embs): Computes the mean and standard deviation of embs and scales it accordingly.
# - np.stack([scaler.mean_, scaler.scale_], axis=1): Stacks the mean and scale values column-wise.
# - torch.tensor(..., dtype=torch.float): Converts the NumPy array into a PyTorch tensor.








# 4. Combine
X_cont = cont                      # (N, E)105
X_categ = specs.astype(int)        # (N, 8)
y = labels.reshape(-1, 1)          # (N, 1)

# 5. Train/test split
Xc_train, Xc_val, Xcat_train, Xcat_val, y_train, y_val = train_test_split(
    X_cont, X_categ, y, test_size=0.2, random_state=42
)

# 6. Dataset & DataLoader
torch.set_num_threads(4)
class TabDataset(Dataset):
    def __init__(self, x_cat, x_cont, y):
        self.x_cat = torch.tensor(x_cat, dtype=torch.long)
        self.x_cont = torch.tensor(x_cont, dtype=torch.float)
        self.y = torch.tensor(y, dtype=torch.float)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return self.x_cat[idx], self.x_cont[idx], self.y[idx]

train_ds = TabDataset(Xcat_train, Xc_train, y_train)
val_ds   = TabDataset(Xcat_val,   Xc_val,   y_val)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# 7. Model instantiation
# category_sizes = # (incorrect—too small!)
# category_sizes = tuple(len(encoder.categories_[i]) for i in range(len(cats))) tuple(len(np.unique(Xcat_train[:, i]))
#                        for i in range(Xcat_train.shape[1]))

category_sizes = tuple(len(encoder.categories_[i]) for i in range(len(cats)))
print("→ category_sizes from encoder:", category_sizes)
#tuple(len(np.unique(Xcat_train[:, i])) for i in range(Xcat_train.shape[1]))
num_cont = Xc_train.shape[1]
model = TabTransformer(
    categories = category_sizes,
    num_continuous = num_cont,
    dim = 128,
    dim_out = 1,
    depth = 6,
    heads = 8,
    attn_dropout = 0.1,
    ff_dropout = 0.1,
    mlp_hidden_mults = (4, 2),
    mlp_act = nn.ReLU(),
    continuous_mean_std = cont_mean_std
).to(DEVICE)

# 8. Loss, optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 9. Training loop
def train_epoch():
    model.train()
    total_loss = 0
    for x_cat, x_cont, y in train_loader:
        x_cat, x_cont, y = x_cat.to(DEVICE), x_cont.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        pred = model(x_cat, x_cont)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * y.size(0)
    return total_loss / len(train_ds)

def eval_epoch():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x_cat, x_cont, y in val_loader:
            x_cat, x_cont, y = x_cat.to(DEVICE), x_cont.to(DEVICE), y.to(DEVICE)
            pred = model(x_cat, x_cont)
            loss = criterion(pred, y)
            total_loss += loss.item() * y.size(0)
    return total_loss / len(val_ds)

# 10. Run training
best_val = float('inf')
for epoch in range(1, NUM_EPOCHS+1):
    train_loss = train_epoch()
    val_loss = eval_epoch()
    print(f"Epoch {epoch:02d}: Train MSE={train_loss:.4f}, Val MSE={val_loss:.4f}")
    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), 'best_tab_transformer.pt')

print("Training complete. Best Val MSE:", best_val)


→ category_sizes from encoder: (3, 29, 21, 4, 2, 5, 17, 9)
Epoch 01: Train MSE=644.0052, Val MSE=283.1101
Epoch 02: Train MSE=254.1086, Val MSE=229.3916
Epoch 03: Train MSE=184.6162, Val MSE=155.7718
Epoch 04: Train MSE=123.8437, Val MSE=120.3899
Epoch 05: Train MSE=98.4093, Val MSE=93.0611
Epoch 06: Train MSE=84.6553, Val MSE=90.5248
Epoch 07: Train MSE=78.5660, Val MSE=87.1688
Epoch 08: Train MSE=73.2374, Val MSE=78.0796
Epoch 09: Train MSE=66.8548, Val MSE=71.5720
Epoch 10: Train MSE=72.0234, Val MSE=71.6430
Epoch 11: Train MSE=64.3342, Val MSE=65.8992
Epoch 12: Train MSE=63.6670, Val MSE=69.5173
Epoch 13: Train MSE=60.1097, Val MSE=63.0226
Epoch 14: Train MSE=56.3550, Val MSE=61.5483
Epoch 15: Train MSE=56.6500, Val MSE=63.4217
Epoch 16: Train MSE=50.8711, Val MSE=49.5062
Epoch 17: Train MSE=46.1923, Val MSE=62.7121
Epoch 18: Train MSE=46.4464, Val MSE=44.4453
Epoch 19: Train MSE=41.9244, Val MSE=52.3081
Epoch 20: Train MSE=39.0633, Val MSE=42.5572
Training complete. Best Val MSE: 